In [3]:
import pandas as pd

In [4]:
base_url = "https://health.data.ny.gov/resource/"
data_tag = 'rmwa-zns4'

In [5]:
base_url + data_tag 

'https://health.data.ny.gov/resource/rmwa-zns4'

In [8]:
facility_count_url = base_url + data_tag + "?" + "$select=facility_id,count(*)&$group=facility_id"

In [9]:
facility_discharge_count_df = pd.read_json(facility_count_url)

In [156]:
facility_discharge_count_df.sort_values("count", ascending=False).head(5)

,count,facility_id
171,56023,1456.0
200,49148,541.0
175,47872,1464.0
172,47136,1458.0
153,43112,1305.0


In [158]:
facility_discharge_count_df["count"].count()

216

In [11]:
def get_socrata_dataset_in_chunks(soda_api_url, limit=10000, order_by=":id"):
    """A simple function to page through results"""
    offset = 0
    data_set_df = None
    
    while True:
        dataset_url =  soda_api_url +  "&$order=" + order_by + "&$offset=" + str(offset) + "&$limit=" + str(limit) + "&$$exclude_system_fields=false"
        print(dataset_url)
        slice_df = pd.read_json(dataset_url)
        record_count = slice_df[order_by].count()
        if data_set_df is None:
            data_set_df = slice_df
        else:
            data_set_df = data_set_df.append(slice_df)
                
        if record_count < limit:
            break
        else:
            data_set_df
        
        offset += limit
        
    return data_set_df     

In [54]:
facility_count_url = base_url + data_tag + "?" + "facility_id=" + str(int(1456))
print(facility_count_url)

https://health.data.ny.gov/resource/rmwa-zns4?facility_id=1456


In [105]:
facility_df = get_socrata_dataset_in_chunks(facility_count_url)

https://health.data.ny.gov/resource/rmwa-zns4?facility_id=1456&$order=:id&$offset=0&$limit=10000&$$exclude_system_fields=false
https://health.data.ny.gov/resource/rmwa-zns4?facility_id=1456&$order=:id&$offset=10000&$limit=10000&$$exclude_system_fields=false
https://health.data.ny.gov/resource/rmwa-zns4?facility_id=1456&$order=:id&$offset=20000&$limit=10000&$$exclude_system_fields=false
https://health.data.ny.gov/resource/rmwa-zns4?facility_id=1456&$order=:id&$offset=30000&$limit=10000&$$exclude_system_fields=false
https://health.data.ny.gov/resource/rmwa-zns4?facility_id=1456&$order=:id&$offset=40000&$limit=10000&$$exclude_system_fields=false
https://health.data.ny.gov/resource/rmwa-zns4?facility_id=1456&$order=:id&$offset=50000&$limit=10000&$$exclude_system_fields=false


In [ ]:
facility_df[":id"].count()

In [ ]:
facility_df.head(5)

In [ ]:
facility_df.columns

In [41]:
facility_df.columns

Index([                        u':created_at',
                             u':created_meta',
                                       u':id',
                               u':updated_at',
                             u':updated_meta',
                   u'abortion_edit_indicator',
                         u'admit_day_of_week',
                                 u'age_group',
                              u'apr_drg_code',
                       u'apr_drg_description',
                              u'apr_mdc_code',
                       u'apr_mdc_description',
          u'apr_medical_surgical_description',
                     u'apr_risk_of_mortality',
              u'apr_severity_of_illness_code',
       u'apr_severity_of_illness_description',
         u'attending_provider_license_number',
                              u'birth_weight',
                        u'ccs_diagnosis_code',
                 u'ccs_diagnosis_description',
                        u'ccs_procedure_code',
             

In [14]:
def create_composite_fields_with_id(df, code, description, padding=4, field_suffix="_with_description"):
    zero_padding = "0" * padding
    df[code + "_with_description"] = df.apply(
    lambda x: str(zero_padding + str(int(x[code])))[-padding:] + " - " + x[description], axis=1)
    return df
    

In [15]:
facility_df_1456 =  create_composite_fields_with_id(facility_df_1456, "apr_drg_code", "apr_drg_description", padding=3)

In [51]:
def prepare_sparcs_df(sparcs_df):
    sparcs_df["length_of_stay_number"] =  sparcs_df.apply(
        lambda x: 120 if "120 +" == x["length_of_stay"] else int(x["length_of_stay"]), axis=1)
    sparcs_df = create_composite_fields_with_id(sparcs_df, "facility_id", "facility_name")
    spracs_df = create_composite_fields_with_id(sparcs_df, "apr_drg_code", "apr_drg_description")
    sparcs_df = create_composite_fields_with_id(sparcs_df, "apr_mdc_code", "apr_mdc_description")
    sparcs_df["in_hospital_mortality"] = sparcs_df["patient_disposition"] == "Expired"

    return sparcs_df

In [106]:
facility_df = prepare_sparcs_df(facility_df)

In [107]:
facility_df["length_of_stay_number"].head()

0      1
1      1
2      1
3    120
4    120
Name: length_of_stay_number, dtype: int64

In [18]:
import numpy as np

In [38]:
def percentile_25th(x):
    return np.percentile(x,0.25)

def percentile_75th(x):
    return np.percentile(x, 0.75)

def percentile_5th(x):
    return np.percentile(x, 0.05)

def percentile_95th(x):
    return np.percentile(x, 0.95)

In [108]:
apr_drg_stats_df_1 = facility_df.groupby(["apr_drg_code_with_description"]).agg(
    {":id": [np.size], "in_hospital_mortality": [np.sum], 
     "length_of_stay_number": [np.mean, np.sum, np.median, 
                               percentile_25th, percentile_75th, percentile_5th, percentile_95th],
    }).reset_index()

In [109]:
apr_drg_stats_df_1[":id"].count()

size    310
dtype: int64

In [110]:
apr_drg_stats_df_2 = pd.crosstab(facility_df["apr_drg_code_with_description"], facility_df_1456["age_group"]).reset_index()

In [111]:
apr_drg_stats_df_3 = pd.merge(apr_drg_stats_df_1, apr_drg_stats_df_2, on="apr_drg_code_with_description", how="outer")

In [112]:
apr_drg_stats_df_4 = pd.crosstab(facility_df["apr_drg_code_with_description"], 
                                 facility_df["apr_severity_of_illness_code"]).reset_index()

In [113]:
apr_drg_stats_df_3.head()

,apr_drg_code_with_description,"(apr_drg_code_with_description, )","(length_of_stay_number, mean)","(length_of_stay_number, sum)","(length_of_stay_number, median)","(length_of_stay_number, percentile_25th)","(length_of_stay_number, percentile_75th)","(length_of_stay_number, percentile_5th)","(length_of_stay_number, percentile_95th)","(:id, size)","(in_hospital_mortality, sum)",0 to 17,18 to 29,30 to 49,50 to 69,70 or Older
0,0001 - LIVER TRANSPLANT &/OR INTESTINAL TRANSP...,0001 - LIVER TRANSPLANT &/OR INTESTINAL TRANSP...,32.522936,3545,20.0,4.5400,5.620,4.1080,6.000,109,6.0,9,3,20,72,5
1,0002 - HEART &/OR LUNG TRANSPLANT,0002 - HEART &/OR LUNG TRANSPLANT,44.275862,1284,22.0,11.0700,11.210,11.0140,11.266,29,2.0,2,0,7,18,2
2,0003 - BONE MARROW TRANSPLANT,0003 - BONE MARROW TRANSPLANT,22.664835,4125,18.0,7.4525,8.715,7.0905,9.439,182,5.0,0,10,33,128,11
3,0004 - TRACHEOSTOMY W MV 96+ HOURS W EXTENSIVE...,0004 - TRACHEOSTOMY W MV 96+ HOURS W EXTENSIVE...,51.898089,8148,45.0,2.0000,2.680,2.0000,3.928,157,45.0,9,3,15,70,60
4,0005 - TRACHEOSTOMY W MV 96+ HOURS W/O EXTENSI...,0005 - TRACHEOSTOMY W MV 96+ HOURS W/O EXTENSI...,46.927419,5819,36.0,5.6150,6.845,5.1230,7.674,124,27.0,2,5,15,41,61


In [114]:
apr_drg_stats_df_5 = pd.merge(apr_drg_stats_df_3, apr_drg_stats_df_4, on="apr_drg_code_with_description", how="outer")

In [116]:
apr_drg_stats_df_6 = pd.crosstab(facility_df["apr_drg_code_with_description"], facility_df["apr_risk_of_mortality"]).reset_index()

In [117]:
apr_drg_stats_df_7 = pd.merge(apr_drg_stats_df_5, apr_drg_stats_df_6, on="apr_drg_code_with_description", how="outer")

In [118]:
apr_drg_stats_df_7.head(5)

,apr_drg_code_with_description,"(apr_drg_code_with_description, )","(length_of_stay_number, mean)","(length_of_stay_number, sum)","(length_of_stay_number, median)","(length_of_stay_number, percentile_25th)","(length_of_stay_number, percentile_75th)","(length_of_stay_number, percentile_5th)","(length_of_stay_number, percentile_95th)","(:id, size)",...,70 or Older,0,1,2,3,4,Extreme,Major,Minor,Moderate
0,0001 - LIVER TRANSPLANT &/OR INTESTINAL TRANSP...,0001 - LIVER TRANSPLANT &/OR INTESTINAL TRANSP...,32.522936,3545,20.0,4.5400,5.620,4.1080,6.000,109,...,5,0,1,10,31,67,54.0,36.0,4.0,15.0
1,0002 - HEART &/OR LUNG TRANSPLANT,0002 - HEART &/OR LUNG TRANSPLANT,44.275862,1284,22.0,11.0700,11.210,11.0140,11.266,29,...,2,0,0,0,11,18,12.0,15.0,0.0,2.0
2,0003 - BONE MARROW TRANSPLANT,0003 - BONE MARROW TRANSPLANT,22.664835,4125,18.0,7.4525,8.715,7.0905,9.439,182,...,11,0,50,91,29,12,15.0,7.0,120.0,40.0
3,0004 - TRACHEOSTOMY W MV 96+ HOURS W EXTENSIVE...,0004 - TRACHEOSTOMY W MV 96+ HOURS W EXTENSIVE...,51.898089,8148,45.0,2.0000,2.680,2.0000,3.928,157,...,60,0,0,0,6,151,134.0,19.0,1.0,3.0
4,0005 - TRACHEOSTOMY W MV 96+ HOURS W/O EXTENSI...,0005 - TRACHEOSTOMY W MV 96+ HOURS W/O EXTENSI...,46.927419,5819,36.0,5.6150,6.845,5.1230,7.674,124,...,61,0,0,0,11,113,96.0,26.0,0.0,2.0


In [119]:
apr_drg_stats_df_8 =  pd.crosstab(facility_df["apr_drg_code_with_description"], facility_df["gender"]).reset_index()

In [121]:
apr_drg_stats_df_9 = pd.merge(apr_drg_stats_df_7, apr_drg_stats_df_8, on="apr_drg_code_with_description", how="outer")

In [122]:
apr_drg_stats_df_9.head(10)

,apr_drg_code_with_description,"(apr_drg_code_with_description, )","(length_of_stay_number, mean)","(length_of_stay_number, sum)","(length_of_stay_number, median)","(length_of_stay_number, percentile_25th)","(length_of_stay_number, percentile_75th)","(length_of_stay_number, percentile_5th)","(length_of_stay_number, percentile_95th)","(:id, size)",...,1,2,3,4,Extreme,Major,Minor,Moderate,F,M
0,0001 - LIVER TRANSPLANT &/OR INTESTINAL TRANSP...,0001 - LIVER TRANSPLANT &/OR INTESTINAL TRANSP...,32.522936,3545,20.0,4.5400,5.6200,4.1080,6.0000,109,...,1,10,31,67,54.0,36.0,4.0,15.0,34,75
1,0002 - HEART &/OR LUNG TRANSPLANT,0002 - HEART &/OR LUNG TRANSPLANT,44.275862,1284,22.0,11.0700,11.2100,11.0140,11.2660,29,...,0,0,11,18,12.0,15.0,0.0,2.0,5,24
2,0003 - BONE MARROW TRANSPLANT,0003 - BONE MARROW TRANSPLANT,22.664835,4125,18.0,7.4525,8.7150,7.0905,9.4390,182,...,50,91,29,12,15.0,7.0,120.0,40.0,83,99
3,0004 - TRACHEOSTOMY W MV 96+ HOURS W EXTENSIVE...,0004 - TRACHEOSTOMY W MV 96+ HOURS W EXTENSIVE...,51.898089,8148,45.0,2.0000,2.6800,2.0000,3.9280,157,...,0,0,6,151,134.0,19.0,1.0,3.0,69,88
4,0005 - TRACHEOSTOMY W MV 96+ HOURS W/O EXTENSI...,0005 - TRACHEOSTOMY W MV 96+ HOURS W/O EXTENSI...,46.927419,5819,36.0,5.6150,6.8450,5.1230,7.6740,124,...,0,0,11,113,96.0,26.0,0.0,2.0,59,65
5,0006 - PANCREAS TRANSPLANT,0006 - PANCREAS TRANSPLANT,13.500000,27,13.5,9.0225,9.0675,9.0045,9.0855,2,...,0,1,0,1,1.0,0.0,0.0,1.0,2,0
6,0020 - CRANIOTOMY FOR TRAUMA,0020 - CRANIOTOMY FOR TRAUMA,5.933333,89,4.0,1.0000,1.0000,1.0000,1.0000,15,...,4,0,6,5,5.0,5.0,1.0,4.0,5,10
7,0021 - CRANIOTOMY EXCEPT FOR TRAUMA,0021 - CRANIOTOMY EXCEPT FOR TRAUMA,6.386364,2810,3.0,1.0000,1.0000,1.0000,1.0000,440,...,110,116,117,97,67.0,120.0,185.0,68.0,231,209
8,0022 - VENTRICULAR SHUNT PROCEDURES,0022 - VENTRICULAR SHUNT PROCEDURES,6.270833,301,4.5,1.0000,1.0000,1.0000,1.0000,48,...,15,17,11,5,3.0,4.0,33.0,8.0,21,27
9,0023 - SPINAL PROCEDURES,0023 - SPINAL PROCEDURES,6.148148,498,5.0,1.0000,1.0000,1.0000,1.0000,81,...,12,33,31,5,4.0,11.0,34.0,32.0,32,49


In [142]:
facility_id_with_description = facility_df["facility_id_with_description"].as_matrix()[0]

In [139]:
discharge_year = facility_df["discharge_year"].as_matrix()[0]

In [140]:
apr_drg_stats_df_9["discharge_year"] = discharge_year

In [143]:
apr_drg_stats_df_9["facility_id_with_description"] = facility_id_with_description

In [144]:
apr_drg_stats_df_9.head(10)

,apr_drg_code_with_description,"(apr_drg_code_with_description, )","(length_of_stay_number, mean)","(length_of_stay_number, sum)","(length_of_stay_number, median)","(length_of_stay_number, percentile_25th)","(length_of_stay_number, percentile_75th)","(length_of_stay_number, percentile_5th)","(length_of_stay_number, percentile_95th)","(:id, size)",...,3,4,Extreme,Major,Minor,Moderate,F,M,discharge_year,facility_id_with_description
0,0001 - LIVER TRANSPLANT &/OR INTESTINAL TRANSP...,0001 - LIVER TRANSPLANT &/OR INTESTINAL TRANSP...,32.522936,3545,20.0,4.5400,5.6200,4.1080,6.0000,109,...,31,67,54.0,36.0,4.0,15.0,34,75,2014,1456 - Mount Sinai Hospital
1,0002 - HEART &/OR LUNG TRANSPLANT,0002 - HEART &/OR LUNG TRANSPLANT,44.275862,1284,22.0,11.0700,11.2100,11.0140,11.2660,29,...,11,18,12.0,15.0,0.0,2.0,5,24,2014,1456 - Mount Sinai Hospital
2,0003 - BONE MARROW TRANSPLANT,0003 - BONE MARROW TRANSPLANT,22.664835,4125,18.0,7.4525,8.7150,7.0905,9.4390,182,...,29,12,15.0,7.0,120.0,40.0,83,99,2014,1456 - Mount Sinai Hospital
3,0004 - TRACHEOSTOMY W MV 96+ HOURS W EXTENSIVE...,0004 - TRACHEOSTOMY W MV 96+ HOURS W EXTENSIVE...,51.898089,8148,45.0,2.0000,2.6800,2.0000,3.9280,157,...,6,151,134.0,19.0,1.0,3.0,69,88,2014,1456 - Mount Sinai Hospital
4,0005 - TRACHEOSTOMY W MV 96+ HOURS W/O EXTENSI...,0005 - TRACHEOSTOMY W MV 96+ HOURS W/O EXTENSI...,46.927419,5819,36.0,5.6150,6.8450,5.1230,7.6740,124,...,11,113,96.0,26.0,0.0,2.0,59,65,2014,1456 - Mount Sinai Hospital
5,0006 - PANCREAS TRANSPLANT,0006 - PANCREAS TRANSPLANT,13.500000,27,13.5,9.0225,9.0675,9.0045,9.0855,2,...,0,1,1.0,0.0,0.0,1.0,2,0,2014,1456 - Mount Sinai Hospital
6,0020 - CRANIOTOMY FOR TRAUMA,0020 - CRANIOTOMY FOR TRAUMA,5.933333,89,4.0,1.0000,1.0000,1.0000,1.0000,15,...,6,5,5.0,5.0,1.0,4.0,5,10,2014,1456 - Mount Sinai Hospital
7,0021 - CRANIOTOMY EXCEPT FOR TRAUMA,0021 - CRANIOTOMY EXCEPT FOR TRAUMA,6.386364,2810,3.0,1.0000,1.0000,1.0000,1.0000,440,...,117,97,67.0,120.0,185.0,68.0,231,209,2014,1456 - Mount Sinai Hospital
8,0022 - VENTRICULAR SHUNT PROCEDURES,0022 - VENTRICULAR SHUNT PROCEDURES,6.270833,301,4.5,1.0000,1.0000,1.0000,1.0000,48,...,11,5,3.0,4.0,33.0,8.0,21,27,2014,1456 - Mount Sinai Hospital
9,0023 - SPINAL PROCEDURES,0023 - SPINAL PROCEDURES,6.148148,498,5.0,1.0000,1.0000,1.0000,1.0000,81,...,31,5,4.0,11.0,34.0,32.0,32,49,2014,1456 - Mount Sinai Hospital


In [145]:
apr_drg_stats_df_9.columns

Index([              u'apr_drg_code_with_description',
              (u'apr_drg_code_with_description', u''),
                  (u'length_of_stay_number', u'mean'),
                   (u'length_of_stay_number', u'sum'),
                (u'length_of_stay_number', u'median'),
       (u'length_of_stay_number', u'percentile_25th'),
       (u'length_of_stay_number', u'percentile_75th'),
        (u'length_of_stay_number', u'percentile_5th'),
       (u'length_of_stay_number', u'percentile_95th'),
                                    (u':id', u'size'),
                   (u'in_hospital_mortality', u'sum'),
                                           u'0 to 17',
                                          u'18 to 29',
                                          u'30 to 49',
                                          u'50 to 69',
                                       u'70 or Older',
                                                    0,
                                                    1,
          

In [155]:
apr_drg_stats_df_9

,apr_drg_code_with_description,"(apr_drg_code_with_description, )","(length_of_stay_number, mean)","(length_of_stay_number, sum)","(length_of_stay_number, median)","(length_of_stay_number, percentile_25th)","(length_of_stay_number, percentile_75th)","(length_of_stay_number, percentile_5th)","(length_of_stay_number, percentile_95th)","(:id, size)",...,3,4,Extreme,Major,Minor,Moderate,F,M,discharge_year,facility_id_with_description
0,0001 - LIVER TRANSPLANT &/OR INTESTINAL TRANSP...,0001 - LIVER TRANSPLANT &/OR INTESTINAL TRANSP...,32.522936,3545,20.0,4.5400,5.6200,4.1080,6.0000,109,...,31,67,54.0,36.0,4.0,15.0,34,75,2014,1456 - Mount Sinai Hospital
1,0002 - HEART &/OR LUNG TRANSPLANT,0002 - HEART &/OR LUNG TRANSPLANT,44.275862,1284,22.0,11.0700,11.2100,11.0140,11.2660,29,...,11,18,12.0,15.0,0.0,2.0,5,24,2014,1456 - Mount Sinai Hospital
2,0003 - BONE MARROW TRANSPLANT,0003 - BONE MARROW TRANSPLANT,22.664835,4125,18.0,7.4525,8.7150,7.0905,9.4390,182,...,29,12,15.0,7.0,120.0,40.0,83,99,2014,1456 - Mount Sinai Hospital
3,0004 - TRACHEOSTOMY W MV 96+ HOURS W EXTENSIVE...,0004 - TRACHEOSTOMY W MV 96+ HOURS W EXTENSIVE...,51.898089,8148,45.0,2.0000,2.6800,2.0000,3.9280,157,...,6,151,134.0,19.0,1.0,3.0,69,88,2014,1456 - Mount Sinai Hospital
4,0005 - TRACHEOSTOMY W MV 96+ HOURS W/O EXTENSI...,0005 - TRACHEOSTOMY W MV 96+ HOURS W/O EXTENSI...,46.927419,5819,36.0,5.6150,6.8450,5.1230,7.6740,124,...,11,113,96.0,26.0,0.0,2.0,59,65,2014,1456 - Mount Sinai Hospital
5,0006 - PANCREAS TRANSPLANT,0006 - PANCREAS TRANSPLANT,13.500000,27,13.5,9.0225,9.0675,9.0045,9.0855,2,...,0,1,1.0,0.0,0.0,1.0,2,0,2014,1456 - Mount Sinai Hospital
6,0020 - CRANIOTOMY FOR TRAUMA,0020 - CRANIOTOMY FOR TRAUMA,5.933333,89,4.0,1.0000,1.0000,1.0000,1.0000,15,...,6,5,5.0,5.0,1.0,4.0,5,10,2014,1456 - Mount Sinai Hospital
7,0021 - CRANIOTOMY EXCEPT FOR TRAUMA,0021 - CRANIOTOMY EXCEPT FOR TRAUMA,6.386364,2810,3.0,1.0000,1.0000,1.0000,1.0000,440,...,117,97,67.0,120.0,185.0,68.0,231,209,2014,1456 - Mount Sinai Hospital
8,0022 - VENTRICULAR SHUNT PROCEDURES,0022 - VENTRICULAR SHUNT PROCEDURES,6.270833,301,4.5,1.0000,1.0000,1.0000,1.0000,48,...,11,5,3.0,4.0,33.0,8.0,21,27,2014,1456 - Mount Sinai Hospital
9,0023 - SPINAL PROCEDURES,0023 - SPINAL PROCEDURES,6.148148,498,5.0,1.0000,1.0000,1.0000,1.0000,81,...,31,5,4.0,11.0,34.0,32.0,32,49,2014,1456 - Mount Sinai Hospital


In [ ]:
# Need to clean the data frame remove replicated columns and rename fields

In [160]:
apr_drg_stats_df_10 = pd.crosstab(facility_df["apr_drg_code_with_description"], facility_df["source_of_payment_1"]).reset_index()

In [161]:
apr_drg_stats_df_10 

source_of_payment_1,apr_drg_code_with_description,Blue Cross/Blue Shield,Federal/State/Local/VA,"Managed Care, Unspecified",Medicaid,Medicare,Miscellaneous/Other,Private Health Insurance,Self-Pay
0,0001 - LIVER TRANSPLANT &/OR INTESTINAL TRANSP...,16,0,1,35,34,0,23,0
1,0002 - HEART &/OR LUNG TRANSPLANT,8,0,0,6,12,0,3,0
2,0003 - BONE MARROW TRANSPLANT,39,0,2,36,45,0,60,0
3,0004 - TRACHEOSTOMY W MV 96+ HOURS W EXTENSIVE...,14,0,2,39,83,1,17,1
4,0005 - TRACHEOSTOMY W MV 96+ HOURS W/O EXTENSI...,5,0,0,27,75,1,16,0
5,0006 - PANCREAS TRANSPLANT,0,0,0,0,2,0,0,0
6,0020 - CRANIOTOMY FOR TRAUMA,1,0,0,1,9,2,2,0
7,0021 - CRANIOTOMY EXCEPT FOR TRAUMA,71,0,4,115,156,4,89,1
8,0022 - VENTRICULAR SHUNT PROCEDURES,7,0,0,13,21,0,7,0
9,0023 - SPINAL PROCEDURES,8,0,1,18,38,4,12,0


In [163]:
apr_drg_stats_df_11 = pd.merge(apr_drg_stats_df_9, apr_drg_stats_df_10, on="apr_drg_code_with_description", how="outer")

In [164]:
apr_drg_stats_11

,apr_drg_code_with_description,"(apr_drg_code_with_description, )","(length_of_stay_number, mean)","(length_of_stay_number, sum)","(length_of_stay_number, median)","(length_of_stay_number, percentile_25th)","(length_of_stay_number, percentile_75th)","(length_of_stay_number, percentile_5th)","(length_of_stay_number, percentile_95th)","(:id, size)",...,discharge_year,facility_id_with_description,Blue Cross/Blue Shield,Federal/State/Local/VA,"Managed Care, Unspecified",Medicaid,Medicare,Miscellaneous/Other,Private Health Insurance,Self-Pay
0,0001 - LIVER TRANSPLANT &/OR INTESTINAL TRANSP...,0001 - LIVER TRANSPLANT &/OR INTESTINAL TRANSP...,32.522936,3545,20.0,4.5400,5.6200,4.1080,6.0000,109,...,2014,1456 - Mount Sinai Hospital,16,0,1,35,34,0,23,0
1,0002 - HEART &/OR LUNG TRANSPLANT,0002 - HEART &/OR LUNG TRANSPLANT,44.275862,1284,22.0,11.0700,11.2100,11.0140,11.2660,29,...,2014,1456 - Mount Sinai Hospital,8,0,0,6,12,0,3,0
2,0003 - BONE MARROW TRANSPLANT,0003 - BONE MARROW TRANSPLANT,22.664835,4125,18.0,7.4525,8.7150,7.0905,9.4390,182,...,2014,1456 - Mount Sinai Hospital,39,0,2,36,45,0,60,0
3,0004 - TRACHEOSTOMY W MV 96+ HOURS W EXTENSIVE...,0004 - TRACHEOSTOMY W MV 96+ HOURS W EXTENSIVE...,51.898089,8148,45.0,2.0000,2.6800,2.0000,3.9280,157,...,2014,1456 - Mount Sinai Hospital,14,0,2,39,83,1,17,1
4,0005 - TRACHEOSTOMY W MV 96+ HOURS W/O EXTENSI...,0005 - TRACHEOSTOMY W MV 96+ HOURS W/O EXTENSI...,46.927419,5819,36.0,5.6150,6.8450,5.1230,7.6740,124,...,2014,1456 - Mount Sinai Hospital,5,0,0,27,75,1,16,0
5,0006 - PANCREAS TRANSPLANT,0006 - PANCREAS TRANSPLANT,13.500000,27,13.5,9.0225,9.0675,9.0045,9.0855,2,...,2014,1456 - Mount Sinai Hospital,0,0,0,0,2,0,0,0
6,0020 - CRANIOTOMY FOR TRAUMA,0020 - CRANIOTOMY FOR TRAUMA,5.933333,89,4.0,1.0000,1.0000,1.0000,1.0000,15,...,2014,1456 - Mount Sinai Hospital,1,0,0,1,9,2,2,0
7,0021 - CRANIOTOMY EXCEPT FOR TRAUMA,0021 - CRANIOTOMY EXCEPT FOR TRAUMA,6.386364,2810,3.0,1.0000,1.0000,1.0000,1.0000,440,...,2014,1456 - Mount Sinai Hospital,71,0,4,115,156,4,89,1
8,0022 - VENTRICULAR SHUNT PROCEDURES,0022 - VENTRICULAR SHUNT PROCEDURES,6.270833,301,4.5,1.0000,1.0000,1.0000,1.0000,48,...,2014,1456 - Mount Sinai Hospital,7,0,0,13,21,0,7,0
9,0023 - SPINAL PROCEDURES,0023 - SPINAL PROCEDURES,6.148148,498,5.0,1.0000,1.0000,1.0000,1.0000,81,...,2014,1456 - Mount Sinai Hospital,8,0,1,18,38,4,12,0


In [170]:
cleaned_columns = []
for col in apr_drg_stats_df_11.columns:
    if col.__class__ == (0, 0).__class__:
        cleaned_columns += ["_".join(col)]
    else:
        cleaned_columns += [col]
apr_drg_stats_df_11.columns = cleaned_columns